In [1]:
from instrument_control import VISA
from instrument_control.INSTRUMENTS import HP_3458A
from pydeveloptools import func_xlsx
from pydeveloptools import func_system
from time import sleep
from enum import Enum
# ----
import ipywidgets as widgets
from IPython.display import display
infobox = widgets.Label(value="¡Bienvenido!")
display(infobox)


Label(value='¡Bienvenido!')

In [2]:
resource_CALIBRATOR = "ASRL2::INSTR"
resource_DMM = "GPIB0::10::INSTR"
resource_COUNTER = "GPIB0::13::INSTR"
NMEAS = 5

# CALIBRATOR
CALIBRATOR = VISA.INSTRUMENT(resource_CALIBRATOR, timeout=10, termination=True)
CALIBRATOR.WR("*CLS")
print(CALIBRATOR.RD("*IDN?"))

# DMM
DMM = HP_3458A.INSTRUMENT(resource_DMM)
DMM.WR("ID?")
print(DMM.RD("A$"))

# COUNTER
COUNTER = VISA.INSTRUMENT(resource_COUNTER, timeout=10, termination=True)
COUNTER.WR("*CLS")
print(COUNTER.RD("*IDN?"))

FLUKE,5080A,3421101,2.03+2.01+2.8
HP3458A
PHILIPS, PM6681, 0, MAIN V1.09  26 JAN 2001 / GPIB V1.13  26 JAN 2001


In [3]:
# HEADER
EXCEL = func_xlsx.XLSREPORT("report")
EXCEL.WR_HEADER(EXCEL.ROW, 1, "REPORT")
EXCEL.WR_HEADER(EXCEL.ROW, 3, func_system.DATE_GET_NOW())
EXCEL.ROW_INC()
EXCEL.LOW_BORDER(EXCEL.ROW)
EXCEL.ROW_INC(3)
EXCEL.SAVE()

In [4]:
from dataclasses import dataclass

@dataclass
class INSTRUMENTS:
    CALIBRATOR: VISA
    DMM: HP_3458A

class TEST:
    '''
    '''
    def __init__(self, instruments: INSTRUMENTS) -> None:
        # INSTRUMENTS
        self.INSTRUMENTS = instruments

    def VDC(self):
        LIST = (
            0.000,
            0.329,
            -0.329,
            0.000,
            1.000,
            -1.000,
            3.290,
            -3.290,
            0.000,
            10.000,
            -10.000,
            32.900,
            -32.900,
            100.000,
            -100.000,
            329.000,
            -329.000,
            330.000,
            -330.000,
            1020.000,
            -1020.000
        )
        # INIT
        self.INSTRUMENTS.CALIBRATOR.WR(f'OUT 0 V')
        self.INSTRUMENTS.DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_DC)


In [5]:
# 1.-MEDIDA DE TENSIÓN CONTINUA. SALIDA NORMAL

def TEST_01():
    # INIT
    CALIBRATOR.WR(f'OUT 0 V')
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_DC)

    LIST = [
        0.000,
        0.329,
        -0.329,
        0.000,
        1.000,
        -1.000,
        3.290,
        -3.290,
        0.000,
        10.000,
        -10.000,
        32.900,
        -32.900,
        100.000,
        -100.000,
        329.000,
        -329.000,
        330.000,
        -330.000,
        1020.000,
        -1020.000
        ]

    for i in LIST:
        print(f"ACQUISITIONS: {i} V")
        CALIBRATOR.WR(f'OUT {i} V')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()

    # FIN
    CALIBRATOR.WR(f'OUT 0 V')
    CALIBRATOR.WR("STBY")

    # 
    EXCEL.SAVE()

In [6]:
# 2.- MEDIDA DE TENSIÓN CONTINUA. SALIDA,UXILAR

def TEST_02():
    # INIT
    CALIBRATOR.WR(f'OUT 0 V')
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_DC)

    LIST = [
        0.000,
        0.329,
        -0.329,
        3.290,
        -3.290,
        7.000,
        -7.000,
    ]

    for i in LIST:
        print(f"ACQUISITIONS: {i} V")
        CALIBRATOR.WR(f'OUT 0V,{i}V')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()

    CALIBRATOR.WR(f'OUT 0 V')
    CALIBRATOR.WR("STBY")

In [7]:
# 3.X- TENSIÓN ALTERNA. SALIDA NORMAL

TEST3_FREQS = [45, 100, 1000]

def TEST_03(frequency = TEST3_FREQS[0]):
    # INIT
    CALIBRATOR.WR(f'OUT 0.01 V, {frequency} HZ')
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_AC)

    LIST = [
        0.0100,
        0.0329,
        0.3290,
        3.2900,
        32.9000,
        100.0000,
        300.0000,
        900.0000,
    ]

    for i in LIST:
        print(f"ACQUISITIONS: {i} V")
        CALIBRATOR.WR(f'OUT {i}V')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 HZ,0 V')
    CALIBRATOR.WR("STBY")

In [8]:
# 4.1- TENSIÓN,LTERNA. SALIDA,UX

TEST4_FREQS = [45, 100, 1000]

def TEST_04(frequency = TEST4_FREQS[0]):
    LIST = [
        0.0100,
        0.3290,
        3.2900,
        5.0000,
    ]
    # INIT
    CALIBRATOR.WR(f'OUT 0.001V,0.01 V,{frequency}HZ')
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_AC)

    for i in LIST:
        print(f"ACQUISITIONS: {i} V")
        CALIBRATOR.WR(f'OUT 0.001V, {i}V')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 HZ,0 V')
    CALIBRATOR.WR("STBY")

In [9]:
# 5.- MEDIDA DE CORRIENTE CONTINUA

def TEST_05():
    # INIT
    CALIBRATOR.WR('OUT 0 HZ')
    CALIBRATOR.WR('OUT 0 A')
    CALIBRATOR.WR('CUR_POST AUX')
    DMM.CONFIG(measure=HP_3458A.MEASURES.CURRENT_DC)

    LIST = [
        0.000000,
        0.000329,
        -0.000329,
        0.000000,
        0.003290,
        -0.003290,
        0.000000,
        0.032900,
        -0.032900,
        0.000000,
        0.329000,
        -0.329000,
        0.000000,
        1.000000,
        -1.000000,
        # 0.000000,
        # 2.990000,
        # -2.990000,
        # 0.000000,
        # 10.900000,
        # -10.900000,
        # 0.000000,
        # 19.990000,
        # -19.990000
        ]

    for i in LIST:
        print(f"ACQUISITIONS: {i} A")
        CALIBRATOR.WR(f'OUT {i} A')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()

    # FIN
    CALIBRATOR.WR(f'OUT 0 A')
    CALIBRATOR.WR("STBY")

    # 
    EXCEL.SAVE()

def TEST_05_SHUNT():
    # INIT
    CALIBRATOR.WR('OUT 0 HZ')
    CALIBRATOR.WR('OUT 0 A')
    CALIBRATOR.WR('CUR_POST A20')
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_DC)

    LIST = [
        # 0.000000,
        # 0.000329,
        # -0.000329,
        # 0.000000,
        # 0.003290,
        # -0.003290,
        # 0.000000,
        # 0.032900,
        # -0.032900,
        # 0.000000,
        # 0.329000,
        # -0.329000,
        # 0.000000,
        # 1.000000,
        # -1.000000,
        0.000000,
        2.990000,
        -2.990000,
        0.000000,
        10.900000,
        -10.900000,
        0.000000,
        19.990000,
        -19.990000
        ]

    for i in LIST:
        print(f"ACQUISITIONS: {i} A")
        CALIBRATOR.WR(f'OUT {i} A')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()

    # FIN
    CALIBRATOR.WR(f'OUT 0 A')
    CALIBRATOR.WR("STBY")

    # 
    EXCEL.SAVE()

In [10]:
# 6.1- CORRIENTE ALTERNA. PRECISIÓN DE PROGRAMACIÓN

TEST6_FREQS = [45, 100, 1000]

def TEST_06(frequency=TEST6_FREQS[0]):
    # INIT
    
    CALIBRATOR.WR('OUT 0.000029 A')
    CALIBRATOR.WR(f'OUT {frequency} HZ')
    CALIBRATOR.WR('CUR_POST AUX')
    DMM.CONFIG(measure=HP_3458A.MEASURES.CURRENT_AC)

    LIST = [
        0.0000290,
        0.0003290,
        0.0032900,
        0.0199000,
        0.1990000,
        1.0000000,
        # 3,
        # 10.0000000,
        # 19.9000000,
        ]

    for i in LIST:
        print(f"ACQUISITIONS: {i} A")
        CALIBRATOR.WR(f'OUT {i} A')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()

    # FIN
    CALIBRATOR.WR(f'OUT 0 A')
    CALIBRATOR.WR("STBY")

    # 
    EXCEL.SAVE()

def TEST_06_SHUNT(frequency=TEST6_FREQS[0]):
    # INIT
    CALIBRATOR.WR("STBY")
    CALIBRATOR.WR('OUT 3 A')
    CALIBRATOR.WR(f'OUT {frequency} HZ')
    CALIBRATOR.WR('CUR_POST A20')
    CALIBRATOR.WR('*WAI')
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_AC)

    LIST = [
        # 0.0000290,
        # 0.0003290,
        # 0.0032900,
        # 0.0199000,
        # 0.1990000,
        # 1.0000000,
        3,
        10.0000000,
        19.9000000,
        ]

    for i in LIST:
        print(f"ACQUISITIONS: {i} A")
        CALIBRATOR.WR(f'OUT {i} A')
        CALIBRATOR.WR('CUR_POST A20')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 1
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()

    # FIN
    CALIBRATOR.WR(f'OUT 0 A')
    CALIBRATOR.WR("STBY")

    # 
    EXCEL.SAVE()

In [11]:
# 7.1- RESISTENCIA	(4W)

def TEST_07():
    LIST = (
        0,
        1,
        1.9,
        10,
        19,
        100,
        190,
        1000,
        1900,
        10000,
        19000,
        100000,
        190000,
    )
    CALIBRATOR.WR('OUT 0 OHM')
    CALIBRATOR.WR('ZCOMP WIRE4')
    DMM.CONFIG(measure=HP_3458A.MEASURES.RESISTANCE_4W)

    for i in LIST:
        print(f"ACQUISITIONS: {i} Ohm")
        CALIBRATOR.WR(f'OUT {i} OHM')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        nominal = CALIBRATOR.RD('OUT?')
        # nominal = float(nominal)
        EXCEL.WR(EXCEL.ROW, 1, nominal)
        DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 HZ,0 V')
    CALIBRATOR.WR("STBY")

In [12]:
# 7.2- RESISTENCIA

def TEST_08():
    LIST = (
        1000000,
        1900000,
        10000000,
        19000000,
        100000000,
        190000000,
    )
    CALIBRATOR.WR('OUT 0 OHM')
    # CALIBRATOR.WR('ZCOMP WIRE2')
    DMM.CONFIG(measure=HP_3458A.MEASURES.RESISTANCE_4W)

    for i in LIST:
        print(f"ACQUISITIONS: {i} Ohm")
        CALIBRATOR.WR(f'OUT {i} OHM')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        nominal = CALIBRATOR.RD('OUT?')
        # nominal = float(nominal)
        EXCEL.WR(EXCEL.ROW, 1, nominal)
        DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 HZ,0 V')
    CALIBRATOR.WR("STBY")

In [13]:
# 9.1- FUENTE BAJA DE RESISTENCIA (LVR)

'''
Paso de buscar los comandos, configuralo a mano
'''

def TEST_09():
    LIST = (
        # LVR ----
        # 1,
        # 1.8,
        # 3.7,
        # 5.9,
        # 10,
        # 18,
        # 37,
        # 59,
        # 100,
        # 180,
        # 370,
        # 590,
        # 1000,
        # 1800,
        # 3700,
        # 5900,
        # HVR ----
        10000,
        11550,
        21000,
        42000,
        80850,
        100000,
        150200,
        288200,
        500000,
        535500,
        1000000,
        1029000,
        1920000,
        3660000,
        6980000,
        10000000,
        10240000,
        20980000,
        39190000,
        76550000,
        100000000,
        138600000,
        148900000,
        289600000,
        559600000,
    )
    DMM.CONFIG(measure=HP_3458A.MEASURES.RESISTANCE_4W)
    
    for i in LIST:
        print(f"ACQUISITIONS: {i} Ohm")
        CALIBRATOR.WR(f'OUT {i} OHM')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        nominal = CALIBRATOR.RD('OUT?')
        # nominal = float(nominal)
        EXCEL.WR(EXCEL.ROW, 1, nominal)
        DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 HZ,0 V')
    CALIBRATOR.WR("STBY")

In [14]:
# 11.1- SCOPE TENSIÓN AC (Carga 1 MΩ)

def TEST_10():
    LIST = (
    (45, 0.005),
    (45, 0.089),
    (45, 0.890),
    (45, 5.000),
    (45, 50.000),
    (100, 0.005),
    (100, 0.020),
    (100, 0.200),
    (100, 1.000),
    (100, 50.000),
    (100, 100.000),
    (1000, 0.005),
    (1000, 0.01),
    (1000, 0.02),
    (1000, 0.05),
    (1000, 0.089),
    (1000, 0.1),
    (1000, 0.2),
    (1000, 0.5),
    (1000, 0.89),
    (1000, 1),
    (1000, 2),
    (1000, 5),
    (1000, 10),
    (1000, 20),
    (1000, 50),
    (1000, 100),
    )
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    CALIBRATOR.WR("SCOPE VOLT")
    # print(CALIBRATOR.RD("SCOPE?"))
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_DC)

    for i in LIST:
        print(f"ACQUISITIONS: {i[0]} Hz, {i[1]} V")
        EXCEL.WR(EXCEL.ROW, 1, f"{i[0]} Hz")
        CALIBRATOR.WR(f'OUT {i[1]}V, {i[0]}HZ;')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            value = value * 2
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    infobox.value = "STOP"

In [15]:
# 11.1- SCOPE TENSIÓN AC (Carga 50 Ω)

def TEST_11():
    LIST = (
        (45, 0.005),
        (45, 0.0449),
        (45, 0.449),
        (45, 2),
        # 
        (100, 0.005),
        (100, 0.01),
        (100, 0.1),
        (100, 1),
        (100, 2),
        # 
        (1000, 0.005),
        (1000, 0.01),
        (1000, 0.02),
        (1000, 0.0449),
        (1000, 0.05),
        (1000, 0.1),
        (1000, 0.2),
        (1000, 0.4999),
        (1000, 0.5),
        (1000, 1),
        (1000, 2),
    )
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    CALIBRATOR.WR("SCOPE VOLT")
    CALIBRATOR.WR("OUT_IMP Z50")
    # print(CALIBRATOR.RD("SCOPE?"))
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_DC)

    for i in LIST:
        print(f"ACQUISITIONS: {i[0]} Hz, {i[1]} V")
        EXCEL.WR(EXCEL.ROW, 1, f"{i[0]} Hz")
        CALIBRATOR.WR(f'OUT {i[1]}V, {i[0]}HZ;')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(3)
        DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            value = value * 2
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    infobox.value = "STOP"

In [16]:
# 12.1- TENSIÓN DC

def TEST_12():
    LIST = (
        # -- 50 OHM --
        # 0,
        # 0.005,
        # -0.005,
        # 0.01,
        # -0.01,
        # 0.022,
        # -0.022,
        # 0.025,
        # -0.025,
        # 0.055,
        # -0.055,
        # 0.1,
        # -0.1,
        # 0.22,
        # -0.22,
        # 0.25,
        # -0.25,
        # 0.55,
        # -0.55,
        # 0.7,
        # -0.7,
        # 2.2,
        # -2.2,
        # -- 1 MOHM --
        0,
        0.005,
        -0.005,
        0.022,
        -0.022,
        0.025,
        -0.025,
        0.045,
        -0.045,
        0.05,
        -0.05,
        0.22,
        -0.22,
        0.25,
        -0.25,
        0.45,
        -0.45,
        0.5,
        -0.5,
        3.3,
        -3.3,
        4,
        -4,
        33,
        -33,
    )
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    CALIBRATOR.WR("SCOPE VOLT")
    # CALIBRATOR.WR("OUT_IMP Z50")
    CALIBRATOR.WR("OUT_IMP Z1M")
    # print(CALIBRATOR.RD("SCOPE?"))
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_DC)

    for i in LIST:
        print(f"ACQUISITIONS: {i} V")
        CALIBRATOR.WR(f'OUT {i}V')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(1)
        DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    infobox.value = "STOP"

In [17]:
# 14.- PRECISIÓN DE LA AMPLITUD. FUNCIÓN LEVELED SINE WAVE

def TEST_13():
    LIST = (
        0.005,
        0.01,
        0.02,
        0.04,
        0.05,
        0.1,
        0.2,
        0.4,
        0.5,
        1.3,
        2,
        5.5,
    )
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    CALIBRATOR.WR("SCOPE LEVSINE")
    CALIBRATOR.WR("OUT_IMP Z50")
    # CALIBRATOR.WR("OUT_IMP Z1M")
    # print(CALIBRATOR.RD("SCOPE?"))
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_AC)

    for i in LIST:
        print(f"ACQUISITIONS: {i} V")
        CALIBRATOR.WR(f'OUT {i}V, 50 KHZ;')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(1)
        DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = DMM.MEAS()
            value = float(value)
            value = value*2**0.5
            value = value * 2
            # vpp: float = (value*2*0,5)*2
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    infobox.value = "STOP"

In [18]:
# 15.- FUNCIÓN DEL GENERADOR DE MARCA

def TEST_14():
    LIST = (
        # 5,
        # 2,
        # 1,
        0.5,
        0.2,
        0.1,
        0.05,
        0.02,
        0.01,
        0.005,
        0.002,
        0.001,
        0.0005,
        0.0002,
        0.0001,
        0.00005,
        0.00002,
        0.00001,
        0.000005,
        0.000002,
        0.000001,
        0.0000005,
        0.0000002,
        0.0000001,
        0.00000005,
        0.00000002,
        0.00000001,
        0.000000005,
        0.000000002,
    )
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    CALIBRATOR.WR("SCOPE MARKER")
    CALIBRATOR.WR("OUT_IMP Z50")
    # CALIBRATOR.WR("OUT_IMP Z1M")
    # print(CALIBRATOR.RD("SCOPE?"))
    DMM.CONFIG(measure=HP_3458A.MEASURES.VOLTAGE_AC)

    for i in LIST:
        print(f"ACQUISITIONS: {i} s")
        CALIBRATOR.WR(f'OUT {i};')
        CALIBRATOR.WR("*CLS")
        CALIBRATOR.WR("OPER")
        CALIBRATOR.WR("*WAI")
        sleep(1)
        # DMM.WR("TARM SGL")
        col = 3
        for meas in range(NMEAS):
            value = COUNTER.RD("READ?;*WAI")
            value = float(value)
            infobox.value = f"({meas+1}) {value}"
            # print(value)
            EXCEL.WR(EXCEL.ROW, col, value)
            col += 1
            sleep(0.5)
        # print()
        EXCEL.ROW_INC()
    
    CALIBRATOR.WR(f'OUT 0 V, 0 HZ;')
    CALIBRATOR.WR("STBY")
    infobox.value = "STOP"

In [19]:
infobox = widgets.Label(value="---")
display(infobox)
# 
TEST_14()

Label(value='---')

ACQUISITIONS: 0.5 s
ACQUISITIONS: 0.2 s
ACQUISITIONS: 0.1 s
ACQUISITIONS: 0.05 s
ACQUISITIONS: 0.02 s
ACQUISITIONS: 0.01 s
ACQUISITIONS: 0.005 s
ACQUISITIONS: 0.002 s
ACQUISITIONS: 0.001 s
ACQUISITIONS: 0.0005 s
ACQUISITIONS: 0.0002 s
ACQUISITIONS: 0.0001 s
ACQUISITIONS: 5e-05 s
ACQUISITIONS: 2e-05 s
ACQUISITIONS: 1e-05 s
ACQUISITIONS: 5e-06 s
ACQUISITIONS: 2e-06 s
ACQUISITIONS: 1e-06 s
ACQUISITIONS: 5e-07 s
ACQUISITIONS: 2e-07 s
ACQUISITIONS: 1e-07 s
ACQUISITIONS: 5e-08 s
ACQUISITIONS: 2e-08 s
ACQUISITIONS: 1e-08 s
ACQUISITIONS: 5e-09 s
ACQUISITIONS: 2e-09 s


In [20]:
# FIN
CALIBRATOR.WR(f'OUT 0 V')
CALIBRATOR.WR("STBY")
CALIBRATOR.CLOSE()
DMM.CLOSE()
EXCEL.SAVE()
EXCEL.CLOSE()
# 
print("FIN")

FIN
